In [ ]:
import math
import json
import librosa
import os
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import files

# Upload the kaggle.json file
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
import os

# Create the .kaggle directory if it doesn't exist
if not os.path.exists("/root/.kaggle"):
    os.makedirs("/root/.kaggle")

# Move the uploaded JSON file to the .kaggle directory
os.rename("kaggle.json", "/root/.kaggle/kaggle.json")

In [ ]:
!kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification

 99% 1.20G/1.21G [00:10<00:00, 141MB/s]
100% 1.21G/1.21G [00:10<00:00, 121MB/s]


In [ ]:
!unzip gtzan-dataset-music-genre-classification

Archive:  gtzan-dataset-music-genre-classification.zip
  inflating: Data/features_30_sec.csv  
  inflating: Data/features_3_sec.csv  
  inflating: Data/genres_original/blues/blues.00000.wav  
  inflating: Data/genres_original/blues/blues.00001.wav  
  inflating: Data/genres_original/blues/blues.00002.wav  
  inflating: Data/genres_original/blues/blues.00003.wav  
  inflating: Data/genres_original/blues/blues.00004.wav  
  inflating: Data/genres_original/blues/blues.00005.wav  
  inflating: Data/genres_original/blues/blues.00006.wav  
  inflating: Data/genres_original/blues/blues.00007.wav  
  inflating: Data/genres_original/blues/blues.00008.wav  
  inflating: Data/genres_original/blues/blues.00009.wav  
  inflating: Data/genres_original/blues/blues.00010.wav  
  inflating: Data/genres_original/blues/blues.00011.wav  
  inflating: Data/genres_original/blues/blues.00012.wav  
  inflating: Data/genres_original/blues/blues.00013.wav  
  inflating: Data/genres_original/blues/blues.00014.wa

In [ ]:
!rm -r gtzan-dataset-music-genre-classification.zip

In [ ]:
def preporcess(dataset_path, num_mfcc=40, n_fft=2048, hop_length=512, num_segment=10):
  data = {"labels": [], "mfcc": []}
  classes = []
  sample_rate = 22050
  samples_per_segment = int(sample_rate*30/num_segment)


  for label_idx, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
    if dirpath == dataset_path:
      continue

    for f in sorted(filenames):
      if not f.endswith('.wav'):
        continue

      file_path = dirpath + "/" + f
      label_name = dirpath.split('/')[-1]
      if label_name not in classes:
        classes.append(label_name)
      print("Track name ", file_path)
      print("LABEL: ", label_idx, label_name)


      try:
        y, sr = librosa.load(file_path, sr = sample_rate)
      except:
        print('ERROR')
        continue
      for n in range(num_segment):
        mfcc = librosa.feature.mfcc(y=y[samples_per_segment*n: samples_per_segment*(n+1)],
                            sr=sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)

        mfcc = mfcc.T
        if len(mfcc) == math.ceil(samples_per_segment/hop_length):
          data["mfcc"].append(mfcc.tolist())
          data["labels"].append(label_idx)
  return data, classes

In [ ]:
mfcc_data, classes = preporcess(dataset_path="/content/Data/genres_original/")

Track name  /content/Data/genres_original/country/country.00000.wav
LABEL:  1 country
Track name  /content/Data/genres_original/country/country.00001.wav
LABEL:  1 country
Track name  /content/Data/genres_original/country/country.00002.wav
LABEL:  1 country
Track name  /content/Data/genres_original/country/country.00003.wav
LABEL:  1 country
Track name  /content/Data/genres_original/country/country.00004.wav
LABEL:  1 country
Track name  /content/Data/genres_original/country/country.00005.wav
LABEL:  1 country
Track name  /content/Data/genres_original/country/country.00006.wav
LABEL:  1 country
Track name  /content/Data/genres_original/country/country.00007.wav
LABEL:  1 country
Track name  /content/Data/genres_original/country/country.00008.wav
LABEL:  1 country
Track name  /content/Data/genres_original/country/country.00009.wav
LABEL:  1 country
Track name  /content/Data/genres_original/country/country.00010.wav
LABEL:  1 country
Track name  /content/Data/genres_original/country/coun

<ipython-input-7-6b136217a8bb>:25: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr = sample_rate)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


ERROR
Track name  /content/Data/genres_original/jazz/jazz.00055.wav
LABEL:  10 jazz
Track name  /content/Data/genres_original/jazz/jazz.00056.wav
LABEL:  10 jazz
Track name  /content/Data/genres_original/jazz/jazz.00057.wav
LABEL:  10 jazz
Track name  /content/Data/genres_original/jazz/jazz.00058.wav
LABEL:  10 jazz
Track name  /content/Data/genres_original/jazz/jazz.00059.wav
LABEL:  10 jazz
Track name  /content/Data/genres_original/jazz/jazz.00060.wav
LABEL:  10 jazz
Track name  /content/Data/genres_original/jazz/jazz.00061.wav
LABEL:  10 jazz
Track name  /content/Data/genres_original/jazz/jazz.00062.wav
LABEL:  10 jazz
Track name  /content/Data/genres_original/jazz/jazz.00063.wav
LABEL:  10 jazz
Track name  /content/Data/genres_original/jazz/jazz.00064.wav
LABEL:  10 jazz
Track name  /content/Data/genres_original/jazz/jazz.00065.wav
LABEL:  10 jazz
Track name  /content/Data/genres_original/jazz/jazz.00066.wav
LABEL:  10 jazz
Track name  /content/Data/genres_original/jazz/jazz.00067.

In [ ]:
classes

['country',
 'metal',
 'blues',
 'classical',
 'rock',
 'disco',
 'reggae',
 'pop',
 'hiphop',
 'jazz']

In [ ]:
import tensorflow as tf

x = np.array(mfcc_data["mfcc"])
y = np.array(mfcc_data["labels"])
y = np.array(y) - 1
print(set(y))

x = x.reshape(x.shape[0], x.shape[1], x.shape[2], 1)
y = tf.keras.utils.to_categorical(y, num_classes=10)


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
x_train, x_val, y_train, y_val =  train_test_split(x_train, y_train, test_size=0.2)

In [ ]:
input_shape = x_train.shape[1:]

In [ ]:
layers = tf.keras.layers

cnn_model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation = 'relu', padding='valid', input_shape=input_shape),
    layers.MaxPooling2D(2, padding="same"),

    layers.Conv2D(64, (3, 3), activation='relu', padding='valid'),
    layers.MaxPooling2D(2, padding="same"),

    layers.Conv2D(128, (3, 3), activation = 'relu', padding='valid', input_shape=input_shape),
    layers.MaxPooling2D(2, padding="same"),
    layers.Dropout(0.3),

    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation = 'relu'),
    layers.Dense(10, activation = 'softmax')
])


In [ ]:
cnn_model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = 'acc')
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 38, 32)       320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 19, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 62, 17, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 31, 9, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 29, 7, 128)        73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 15, 4, 128)        0

In [ ]:
history = cnn_model.fit(x_train, y_train,
                       validation_data=(x_val, y_val),
                       epochs=40,
                       verbose=2,
                       batch_size=32)

Epoch 1/40
188/188 - 11s - loss: 0.2570 - acc: 0.3934 - val_loss: 0.2211 - val_acc: 0.4740 - 11s/epoch - 58ms/step
Epoch 2/40
188/188 - 2s - loss: 0.1947 - acc: 0.5587 - val_loss: 0.1926 - val_acc: 0.5594 - 2s/epoch - 8ms/step
Epoch 3/40
188/188 - 2s - loss: 0.1711 - acc: 0.6258 - val_loss: 0.1775 - val_acc: 0.6155 - 2s/epoch - 8ms/step
Epoch 4/40
188/188 - 2s - loss: 0.1560 - acc: 0.6705 - val_loss: 0.1535 - val_acc: 0.6669 - 2s/epoch - 9ms/step
Epoch 5/40
188/188 - 2s - loss: 0.1392 - acc: 0.7122 - val_loss: 0.1431 - val_acc: 0.7089 - 2s/epoch - 10ms/step
Epoch 6/40
188/188 - 2s - loss: 0.1310 - acc: 0.7344 - val_loss: 0.1246 - val_acc: 0.7517 - 2s/epoch - 8ms/step
Epoch 7/40
188/188 - 2s - loss: 0.1179 - acc: 0.7561 - val_loss: 0.1218 - val_acc: 0.7570 - 2s/epoch - 8ms/step
Epoch 8/40
188/188 - 2s - loss: 0.1071 - acc: 0.7872 - val_loss: 0.1175 - val_acc: 0.7784 - 2s/epoch - 8ms/step
Epoch 9/40
188/188 - 2s - loss: 0.0986 - acc: 0.8057 - val_loss: 0.1068 - val_acc: 0.7897 - 2s/epoch

In [ ]:
y_pred = cnn_model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)

print(np.sum(y_pred==y_test)/len(y_pred))

79/79 [==============================] - 0s 5ms/step
0.882258710452543


In [ ]:
print(np.sum(y_pred==y_test)/len(y_pred))

0.882258710452543


In [ ]:
def predictionprocess(file_path, num_mfcc=40, n_fft=2048, hop_length=512, num_segment=10):
    sample_rate = 22050
    samples_per_track = sample_rate * 30
    samples_per_segment = int(samples_per_track / num_segment)
    data = {"mfcc": []}

    # Load the audio file
    signal, sample_rate = librosa.load(file_path, sr=sample_rate)

    # Process segments extracting mfcc
    for d in range(num_segment):
        start = samples_per_segment * d
        end = start + samples_per_segment

        # Corrected function call with keyword arguments
        mfcc = librosa.feature.mfcc(y=signal[start:end], sr=sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
        mfcc = mfcc.T

        # Store mfcc for segment if it has the expected length
        if len(mfcc) == math.ceil(samples_per_segment / hop_length):
            data["mfcc"].append(mfcc.tolist())

    return data["mfcc"]


In [ ]:
new_song_mfcc = predictionprocess(file_path="/content/audio.mp3")

In [ ]:
new_song_mfcc = np.array(new_song_mfcc)
new_song_mfcc = new_song_mfcc[..., np.newaxis]

In [ ]:
predictions = cnn_model.predict(new_song_mfcc)
predicted_class_indices = np.argmax(predictions, axis=1)

1/1 [==============================] - 0s 20ms/step


In [ ]:
predicted_class_indices


array([3, 8, 8, 8, 5, 5, 5, 5, 5, 5])

In [ ]:
mode = np.bincount(predicted_class_indices).argmax()
classes[mode]

'disco'

In [ ]:
cnn_model.save("cnn_model.h5")